In [183]:
import requests
from bs4 import BeautifulSoup
import pandas
import numpy as np

headers = {
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'  
}

In [260]:
stock_id = 6216

In [261]:
def get_cash_dividend_payouts( stock_id, 
                               year_to_check=5):
    res = requests.get("https://histock.tw/stock/{}/%E9%99%A4%E6%AC%8A%E9%99%A4%E6%81%AF".format(stock_id), headers = headers)
    res.encoding = 'utf-8'
    
    soup = BeautifulSoup(res.text, 'lxml')
    data = soup.select_one("table", class_="tb-outline")

    dfs = pandas.read_html(data.prettify())

    df = dfs[0]
    df = df[1:]
    
    cash_dividend_payouts = df["現金股利"][:year_to_check].astype('float')
    return list(cash_dividend_payouts)

In [262]:
def get_ROEs( stock_id, 
              year_to_check=5):
    
    res = requests.get("https://histock.tw/stock/financial.aspx?no={}&t=3&st=2&q=3".format(stock_id), headers = headers)
    res.encoding = 'utf-8'

    soup = BeautifulSoup(res.text, 'lxml')
    data = soup.select_one("table", class_="tb-outline")

    dfs = pandas.read_html(data.prettify())

    df = dfs[0]
    ROEs = [float(ROE.strip('%')) for ROE in df["年度ROE"][:year_to_check]]
    return ROEs

In [263]:
def get_PEratios( stock_id,
                  year_to_check=20):
    
    res = requests.get("https://histock.tw/stock/{}/%E6%9C%AC%E7%9B%8A%E6%AF%94".format(stock_id), headers = headers)
    res.encoding = 'utf-8'

    soup = BeautifulSoup(res.text, 'lxml')
    data = soup.select_one("table", class_="tb-outline")

    dfs = pandas.read_html(data.prettify())

    df = dfs[0]
    df = df[1:]
    
    PEratio_cols = [col for col in df.columns if '本益比' in col]
    PEratios = []
    for col in PEratio_cols:
        PEratios += list(df[col])

    return PEratios[:year_to_check]

In [264]:
def get_prev_net_val_per_share( stock_id, 
                                prev_year="2019"):

    res = requests.get("https://histock.tw/stock/{}/%E6%AF%8F%E8%82%A1%E6%B7%A8%E5%80%BC".format(stock_id), headers = headers)
    res.encoding = 'utf-8'

    soup = BeautifulSoup(res.text, 'lxml')
    data = soup.select_one("table", class_="tb-outline")
    dfs = pandas.read_html(data.prettify())

    df = dfs[0]

    prev_net_val_per_share = df.iloc[3][prev_year]
    return prev_net_val_per_share

In [265]:
def get_EPSs( stock_id, 
              year_to_check=5):
    
    res = requests.get("https://histock.tw/stock/{}/%E9%99%A4%E6%AC%8A%E9%99%A4%E6%81%AF".format(stock_id), headers = headers)
    res.encoding = 'utf-8'
    
    soup = BeautifulSoup(res.text, 'lxml')
    data = soup.select_one("table", class_="tb-outline")

    dfs = pandas.read_html(data.prettify())

    df = dfs[0]
    df = df[1:]

    EPSs = df["EPS"][:year_to_check].astype('float')
    return list(EPSs)

In [266]:
cash_dividend_payouts = get_cash_dividend_payouts( stock_id, year_to_check=5 )
EPSs = get_EPSs( stock_id, year_to_check=5 )
ROEs = get_ROEs( stock_id, year_to_check=5 )
PEratios = get_PEratios( stock_id, year_to_check=20 )
prev_net_val_per_share = get_prev_net_val_per_share( stock_id, prev_year="2019" )
dividend_payout_ratios = np.array(cash_dividend_payouts) / np.array(EPSs)

In [267]:
# cash_dividend_payouts = np.array()
print (cash_dividend_payouts)
print (EPSs)
print (ROEs)
print (PEratios)
print (dividend_payout_ratios)
print (prev_net_val_per_share)

[1.87, 1.98, 2.06, 2.64, 2.62]
[1.8, 2.28, 2.38, 3.07, 2.99]
[10.09, 12.91, 13.67, 17.38, 17.48]
[17.48, 17.79, 14.84, 15.11, 14.78, 15.48, 13.71, 14.07, 14.12, 14.43, 14.38, 14.79, 14.74, 14.08, 13.86, 13.74, 12.79, 12.83, 11.79, 11.5]
[1.03888889 0.86842105 0.86554622 0.85993485 0.87625418]
17.88


In [268]:
exptect_ROE = np.mean(ROEs) / 100 
print ('{:>36}  {:>4.2f} (year)'.format("exptect_ROE", exptect_ROE ))

exptect_dividend_payout_ratio = np.mean(dividend_payout_ratios) 
print ('{:>36}  {:>4.2f}'.format("exptect_dividend_payout_ratio", exptect_dividend_payout_ratio ))

prev_EPS = EPSs[-1]
prev_dividend_payout = cash_dividend_payouts[-1]
cur_net_val_per_share = prev_net_val_per_share + (prev_EPS - prev_dividend_payout)
print ('{:>36}  {:>4.2f}'.format("cur_net_val_per_share (accumulated)", cur_net_val_per_share ))

cur_EPS = exptect_ROE * cur_net_val_per_share
print ('{:>36}  {:>4.2f}'.format("cur_EPS", cur_EPS ))

cur_dividend_payout = cur_EPS * exptect_dividend_payout_ratio
print ('{:>36}  {:>4.2f}'.format("cur_dividend_payout", cur_dividend_payout ))

EPS_8th = exptect_ROE * prev_net_val_per_share
print ('{:>36}  {:>4.2f}'.format("EPS_8th", EPS_8th ))

expect_PEratio = np.mean(PEratios)
expect_PEratio = min( expect_PEratio, 12 )
print ('{:>36}  {:>4.2f}'.format("expect_PEratio", expect_PEratio ))

sell_price = EPS_8th * expect_PEratio
print ('{:>36}  {:>4.2f}'.format("sell_price", sell_price ))

                         exptect_ROE  0.14 (year)
       exptect_dividend_payout_ratio  0.90
 cur_net_val_per_share (accumulated)  18.25
                             cur_EPS  2.61
                 cur_dividend_payout  2.35
                             EPS_8th  2.56
                      expect_PEratio  12.00
                          sell_price  30.69


In [269]:
buy_price = 0
ror = 0.15 # rate of return = 15%
year_to_hold = 8
for year in range(1, year_to_hold+1):
    if year == year_to_hold:
        buy_price += (cur_dividend_payout + sell_price) / ((1 + ror) ** year )
    else: 
        buy_price += cur_dividend_payout / ((1 + ror) ** year )
    print ("year {} => buy_price: {:.2f}".format(year, buy_price))

print ("buy_price: {:.2f}".format(buy_price))

year 1 => buy_price: 2.05
year 2 => buy_price: 3.83
year 3 => buy_price: 5.38
year 4 => buy_price: 6.72
year 5 => buy_price: 7.89
year 6 => buy_price: 8.91
year 7 => buy_price: 9.80
year 8 => buy_price: 20.60
buy_price: 20.60
